In [47]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [48]:
data_fake = pd.read_csv('/content/Fake.csv')
data_true = pd.read_csv('/content/True.csv')


In [49]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [50]:
data_fake['class'] = 0
data_true['class'] = 1

In [51]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [52]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480, 23470, -1):
  data_fake.drop([i], axis = 0, inplace = True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416, 21406, -1):
  data_true.drop([i], axis = 0, inplace = True)



In [53]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [54]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

<ipython-input-54-90008d39c97b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
<ipython-input-54-90008d39c97b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [56]:
data_merge = pd.concat([data_fake, data_true], axis = 0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [57]:
data_merge.columns


Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [58]:
data = data_merge.drop(['title', 'subject', 'date'], axis = 1)

In [59]:
data.isnull().sum()

,0
text,0
class,0


In [60]:
data = data.sample(frac = 1)
data.head()

,text,class
6398,WASHINGTON (Reuters) - U.S. President-elect Do...,1
4120,NEW YORK (Reuters) - U.S. President Donald Tru...,1
11889,BERLIN (Reuters) - European aviation safety au...,1
14999,JAKARTA (Reuters) - The Indonesian unit of Fre...,1
7241,NEW YORK (Reuters) - U.S. President-elect Dona...,1


In [61]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [62]:
data.columns

Index(['text', 'class'], dtype='object')

In [68]:
def text_processing(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub("\\W", " ", text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return text

In [69]:
data['text'] = data['text'].apply(text_processing)

In [70]:
x = data['text']
y = data['class']


In [71]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xvector_train = vectorization.fit_transform(x_train)
xvector_test = vectorization.transform(x_test)

In [78]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xvector_train, y_train)

LogisticRegression()

In [79]:
pred_lr = LR.predict(xvector_test)

LR.score(xvector_test, y_test)

0.9866310160427807

In [83]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5895
           1       0.98      0.99      0.99      5325

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [80]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xvector_train, y_train)

DecisionTreeClassifier()

In [81]:
pred_dt = DT.predict(xvector_test)

In [82]:
DT.score(xvector_test, y_test)

0.9962566844919786

In [85]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5895
           1       1.00      0.99      1.00      5325

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [87]:
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xvector_train, y_train)

GradientBoostingClassifier(random_state=0)

In [88]:
pred_gb = GB.predict(xvector_test)

In [89]:
GB.score(xvector_test, y_test)

0.9952762923351158

In [90]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5895
           1       0.99      1.00      1.00      5325

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [91]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state = 0)
RF.fit(xvector_train, y_train)

RandomForestClassifier(random_state=0)

In [92]:
pred_rf = RF.predict(xvector_test)

In [93]:
RF.score(xvector_test, y_test)

0.988680926916221

In [94]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.99      0.99      0.99      5325

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [101]:
def output_label(n):
  if n ==0:
    return "Fake News"
  if n == 1:
    return "True News"


def testing(news):
  testing_news = {"text":[news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(text_processing)
  new_x_test = new_def_test["text"]
  new_xvector_test = vectorization.transform(new_x_test)
  pred_LR = LR.predict(new_xvector_test)
  pred_DT = DT.predict(new_xvector_test)
  pred_GB = GB.predict(new_xvector_test)
  pred_RF = RF.predict(new_xvector_test)

  return print("\n\n LR Prediction: {} \n DT predictiion: {} \n GB prediction: {} \n RF prediction: {}".format(output_label(pred_LR[0]),
                                                                                                               output_label(pred_DT[0]),
                                                                                                               output_label(pred_GB[0]),
                                                                                                               output_label(pred_RF[0])))

In [107]:
news = str(input())
testing(news)

IBM is quietly laying off thousands of employees, mainly impacting senior-level programmers, sales and support staff. The recent cuts were carried out in secret with employees sharing they were asked to sign an NDA (Non-Disclosure Agreement), and not talk about the specifics, per a report by The Register.  The company reportedly took a $400 million “workforce rebalancing” charge to cover the cost of planned layoffs in the first quarter of 2024. The charge follows a similar spending of $300 million in 2023. IBM had announced plans to cut 3,900 jobs in 2023.  IBM’s spokesperson shared that the cost of workforce rebalancing represents a very low single digit percentage of the company’s workforce, and that it expects to exit 2024 with roughly the same number of employees as the beginning of the year.  However, the company did not share the exact number of employees affected by the layoffs.  The company had about 288,000 employees worldwide at the end of 2023, and with the fixed cost per em